In [ ]:
# Initialization: Load shared functions and simulated data 

# Load shared functions
!curl -O https://raw.githubusercontent.com/Fraud-Detection-Handbook/fraud-detection-handbook/main/Chapter_References/shared_functions.py
%run shared_functions.py

# Get simulated data from Github repository
if not os.path.exists("simulated-data-transformed"):
    !git clone https://github.com/Fraud-Detection-Handbook/simulated-data-transformed
        

In [ ]:
# Load data from the 2018-07-25 to the 2018-08-14

dir_input = './simulated-data-transformed/data/' 

begin_date = "2018-07-25"   # 2023-12-01 in other dataset I created
end_date = "2018-08-14"     # 2024-02-28 in other dataset I created

print("Load  files")
%time transactions_df = read_from_files(dir_input, begin_date, end_date)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.tx_fraud.sum()))

In [ ]:
# Compute the number of transactions per day, fraudulent transactions per day and fraudulent cards per day

def get_tx_stats(transactions_df, start_date_df='2018-04-01'):  # start_date_df='2023-12-01' from other dataset I created
    
    #Number of transactions per day
    nb_tx_per_day = transactions_df.groupby(['tx_time_days'])['customer_id'].count()
    #Number of fraudulent transactions per day
    nb_fraudulent_transactions_per_day = transactions_df.groupby(['tx_time_days'])['tx_fraud'].sum()
    #Number of compromised cards per day
    nb_compromised_cards_per_day = transactions_df[transactions_df['TX_FRAUD']==1].groupby(['tx_time_days']).customer_id.nunique()
    
    tx_stats = pd.DataFrame({"nb_tx_per_day":nb_tx_per_day,
                           "nb_fraudulent_transactions_per_day":nb_fraudulent_transactions_per_day,
                           "nb_compromised_cards_per_day":nb_compromised_cards_per_day})

    tx_stats = tx_stats.reset_index()
    
    start_date = datetime.datetime.strptime(start_date_df, "%Y-%M-%D")
    tx_date = start_date+tx_stats['tx_time_days'].apply(datetime.timedelta)
    
    tx_stats['tx_date'] = tx_date
    
    return tx_stats

tx_stats=get_tx_stats(transactions_df, start_date_df="2018-04-01")